In [ ]:
# Step 1: Import Required Libraries
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from google.colab import files
print("Libraries imported.")

Libraries imported.


In [ ]:
import pandas as pd

# Create small fake datasets similar to the Book-Crossings structure
ratings_data = {
    'User-ID': [1,1,1,2,2,2,3,3,3,4,4,4,5,5,5],
    'ISBN': ['A','B','C','A','B','D','A','C','D','B','C','E','A','D','E'],
    'Book-Rating': [10,9,8,9,7,6,8,10,9,7,6,8,9,8,7]
}

books_data = {
    'ISBN': ['A','B','C','D','E'],
    'Book-Title': [
        'The Queen of the Damned (Vampire Chronicles (Paperback))',
        'Interview with the Vampire',
        'The Witching Hour (Lives of the Mayfair Witches)',
        'The Vampire Lestat (Vampire Chronicles, Book II)',
        'The Tale of the Body Thief (Vampire Chronicles (Paperback))'
    ]
}

# Convert to pandas DataFrames
ratings = pd.DataFrame(ratings_data)
books = pd.DataFrame(books_data)

print("✅ Sample dataset created successfully!")
print("Ratings:\n", ratings.head(), "\n")
print("Books:\n", books.head())


✅ Sample dataset created successfully!
Ratings:
    User-ID ISBN  Book-Rating
0        1    A           10
1        1    B            9
2        1    C            8
3        2    A            9
4        2    B            7 

Books:
   ISBN                                         Book-Title
0    A  The Queen of the Damned (Vampire Chronicles (P...
1    B                         Interview with the Vampire
2    C   The Witching Hour (Lives of the Mayfair Witches)
3    D   The Vampire Lestat (Vampire Chronicles, Book II)
4    E  The Tale of the Body Thief (Vampire Chronicles...


In [ ]:
# Step 3: Prepare dataset (filter + pivot)

# (Since this is small sample data, we won't filter by 100/200 ratings)

# Create pivot table: rows = ISBN, columns = User-ID, values = Book-Rating
book_pivot = ratings.pivot(index='ISBN', columns='User-ID', values='Book-Rating').fillna(0)

print("✅ Pivot table created successfully!")
print(book_pivot)


✅ Pivot table created successfully!
User-ID     1    2     3    4    5
ISBN                              
A        10.0  9.0   8.0  0.0  9.0
B         9.0  7.0   0.0  7.0  0.0
C         8.0  0.0  10.0  6.0  0.0
D         0.0  6.0   9.0  0.0  8.0
E         0.0  0.0   0.0  8.0  7.0


In [ ]:
# Step 4: Fit the KNN model and create the recommender function
from sklearn.neighbors import NearestNeighbors

# Fit model using cosine similarity
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(book_pivot.values)

# Mapping between ISBN and Book Titles
book_titles = books.set_index('ISBN')['Book-Title'].to_dict()

# Create function to get book recommendations
def get_recommends(book_title):
    # Find ISBN by book title
    isbn = None
    for k, v in book_titles.items():
        if v == book_title:
            isbn = k
            break

    if not isbn:
        return f"❌ Book '{book_title}' not found."

    # Get the row index in the pivot table
    book_idx = book_pivot.index.get_loc(isbn)
     # Find 6 nearest neighbors (1 original + 5 recommendations)
    distances, indices = model.kneighbors(book_pivot.iloc[book_idx, :].values.reshape(1, -1), n_neighbors=5)


    # Create result list
    recommended_books = [
        [book_titles[book_pivot.index[indices.flatten()[i]]], round(float(distances.flatten()[i]), 3)]
        for i in range(1, len(distances.flatten()))
    ]

    return [book_title, recommended_books]


In [ ]:
result = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(result)


['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Vampire Lestat (Vampire Chronicles, Book II)', 0.185], ['Interview with the Vampire', 0.367], ['The Witching Hour (Lives of the Mayfair Witches)', 0.373], ['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.672]]]
